In [1]:
pwd

'/home/jovyan/work'

In [2]:
import pandas as pd
import linecache

In [3]:
targz_contents_fn = 'data/GHCN_20191001/daily-summaries-directory.txt'
stations_fn = 'data/GHCN_20191001/ghcnd-stations.txt'

In [4]:
linecache.getline(targz_contents_fn, 10)

'-rw-r--r--  0 0      0       64912 Oct  1 07:44 USC00214066.csv\n'

In [5]:
linecache.getline(stations_fn, 114)

'AJ000037742  40.9000   47.3000  313.0    ORDJONIKIDZE,ZERNOSOVHOZ               37742\n'

In [6]:
directory_header = ['type_permissions', 'num_links', 'owner', 'group', 'size_bytes', 'month', 'day', 'time', 'fn']

In [7]:
data_dirs = pd.read_csv(targz_contents_fn, delim_whitespace=True, header=None, names=directory_header)

In [8]:
data_dirs.head()

,type_permissions,num_links,owner,group,size_bytes,month,day,time,fn
0,-rw-r--r--,0,0,0,72466,Oct,1,07:44,USC00169027.csv
1,-rw-r--r--,0,0,0,371181,Oct,1,07:44,USC00166678.csv
2,-rw-r--r--,0,0,0,1072270,Oct,1,07:44,USC00166466.csv
3,-rw-r--r--,0,0,0,86581,Oct,1,07:44,USC00172883.csv
4,-rw-r--r--,0,0,0,3613726,Oct,1,07:44,USC00193985.csv


In [9]:
data_dirs.count()

type_permissions    114781
num_links           114781
owner               114781
group               114781
size_bytes          114781
month               114781
day                 114781
time                114781
fn                  114781
dtype: int64

In [10]:
data_dirs['fn_prefix'] = data_dirs.fn.str[0:3]

In [11]:
data_dirs.head()

,type_permissions,num_links,owner,group,size_bytes,month,day,time,fn,fn_prefix
0,-rw-r--r--,0,0,0,72466,Oct,1,07:44,USC00169027.csv,USC
1,-rw-r--r--,0,0,0,371181,Oct,1,07:44,USC00166678.csv,USC
2,-rw-r--r--,0,0,0,1072270,Oct,1,07:44,USC00166466.csv,USC
3,-rw-r--r--,0,0,0,86581,Oct,1,07:44,USC00172883.csv,USC
4,-rw-r--r--,0,0,0,3613726,Oct,1,07:44,USC00193985.csv,USC


In [12]:
unique_data_dirs = data_dirs['fn_prefix'].value_counts()

In [13]:
unique_data_dirs.head(40)

US1    35020
USC    22480
ASN    17081
CA0     7901
BR0     5934
MXN     5170
IN0     3805
USW     1742
SWE     1703
USR     1509
SF0     1154
RSM     1102
GME      967
FIE      917
CA1      876
USS      859
NOE      438
NLE      381
KZ0      328
WA0      281
CHM      226
UPM      204
SPE      183
JA0      154
RQC      151
UY0      146
UKE      108
GG0      102
GMM       94
IDM       87
AYM       87
VE0       81
MXM       79
UZM       78
KG0       73
AJ0       66
AGM       63
TI0       62
ARM       61
ITE       58
Name: fn_prefix, dtype: int64

In [14]:
len(unique_data_dirs)

421

In [15]:
unique_data_dirs.head(20).sum()

109648

In [17]:
unique_data_dirs.head(20)

US1    35020
USC    22480
ASN    17081
CA0     7901
BR0     5934
MXN     5170
IN0     3805
USW     1742
SWE     1703
USR     1509
SF0     1154
RSM     1102
GME      967
FIE      917
CA1      876
USS      859
NOE      438
NLE      381
KZ0      328
WA0      281
Name: fn_prefix, dtype: int64

In [18]:
# We used fixed width here due to the specification from 
# https://www1.ncdc.noaa.gov/pub/data/ghcn/daily/readme.txt

ghcnd_stations_format =[
    ('ID', 0, 11, 'str'),
    ('LATITUDE', 12, 21, 'float'),
    ('LONGITUDE', 21, 30, 'float'),
    ('ELEVATION', 31, 37, 'float'),
    ('STATE', 38, 40, 'str'),
    ('NAME', 41, 71, 'str'),
    ('GSN FLAG', 72, 75, 'str'),
    ('HCN/CRN FLAG', 76, 79, 'str'),
    ('WMO ID', 80, 85, 'str'),
]
ghcnd_colspecs = []
ghcnd_names =[]
ghcnd_dtypes =[]
for name, start, stop, dtype in ghcnd_stations_format:
    ghcnd_colspecs.append((start, stop))
    ghcnd_names.append(name.lower())
    ghcnd_dtypes.append(dtype)
ghcnd_name_type_dict = dict(zip(ghcnd_names, ghcnd_dtypes))                            

In [19]:
ghcnd_stations = pd.read_fwf(stations_fn, 
                             colspecs=ghcnd_colspecs,
                             header=None,
                             names=ghcnd_names,
                             dtype=ghcnd_name_type_dict
                            )

In [20]:
len(ghcnd_stations)

114789

In [16]:
114789-109648

5141

In [21]:
us1_stations = ghcnd_stations[ghcnd_stations['id'].str.contains('US1')]
us1_stations

,id,latitude,longitude,elevation,state,name,gsn flag,hcn/crn flag,wmo id
52397,US10RMHS145,40.5268,-105.1113,1569.1,CO,RMHS 1.6 SSW,NaN,NaN,NaN
52398,US10adam001,40.5680,-98.5069,598.0,NE,JUNIATA 1.5 S,NaN,NaN,NaN
52399,US10adam002,40.5093,-98.5493,601.1,NE,JUNIATA 6.0 SSW,NaN,NaN,NaN
52400,US10adam003,40.4663,-98.6537,615.1,NE,HOLSTEIN 0.1 NW,NaN,NaN,NaN
52401,US10adam004,40.4798,-98.4026,570.0,NE,AYR 3.5 NE,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
87412,US1WYWS0024,43.8575,-104.1402,1403.6,WY,NEWCASTLE 3.5 E,NaN,NaN,NaN
87413,US1WYWS0025,43.8303,-104.2333,1290.2,WY,NEWCASTLE 1.7 SW,NaN,NaN,NaN
87414,US1WYWS0026,44.1153,-104.6278,1314.0,WY,UPTON 1.0 NNW,NaN,NaN,NaN
87415,US1WYWS0029,43.7019,-104.6947,1326.8,WY,NEWCASTLE 26.2 WSW,NaN,NaN,NaN


### Visualize location of a subset of these stations on [Map Maker](https://maps.co)

In [22]:
mapmaker_us1 = us1_stations[['latitude', 'longitude', 'name', 'id']].copy()
mapmaker_us1['color'] = '#FFFF00'
mapmaker_us1

,latitude,longitude,name,id,color
52397,40.5268,-105.1113,RMHS 1.6 SSW,US10RMHS145,#FFFF00
52398,40.5680,-98.5069,JUNIATA 1.5 S,US10adam001,#FFFF00
52399,40.5093,-98.5493,JUNIATA 6.0 SSW,US10adam002,#FFFF00
52400,40.4663,-98.6537,HOLSTEIN 0.1 NW,US10adam003,#FFFF00
52401,40.4798,-98.4026,AYR 3.5 NE,US10adam004,#FFFF00
...,...,...,...,...,...
87412,43.8575,-104.1402,NEWCASTLE 3.5 E,US1WYWS0024,#FFFF00
87413,43.8303,-104.2333,NEWCASTLE 1.7 SW,US1WYWS0025,#FFFF00
87414,44.1153,-104.6278,UPTON 1.0 NNW,US1WYWS0026,#FFFF00
87415,43.7019,-104.6947,NEWCASTLE 26.2 WSW,US1WYWS0029,#FFFF00


In [23]:
mapmaker_us1 = mapmaker_us1.rename(columns={'id':'note'})
mapmaker_us1 = mapmaker_us1[['latitude', 'longitude', 'name', 'color', 'note']]
mapmaker_us1

,latitude,longitude,name,color,note
52397,40.5268,-105.1113,RMHS 1.6 SSW,#FFFF00,US10RMHS145
52398,40.5680,-98.5069,JUNIATA 1.5 S,#FFFF00,US10adam001
52399,40.5093,-98.5493,JUNIATA 6.0 SSW,#FFFF00,US10adam002
52400,40.4663,-98.6537,HOLSTEIN 0.1 NW,#FFFF00,US10adam003
52401,40.4798,-98.4026,AYR 3.5 NE,#FFFF00,US10adam004
...,...,...,...,...,...
87412,43.8575,-104.1402,NEWCASTLE 3.5 E,#FFFF00,US1WYWS0024
87413,43.8303,-104.2333,NEWCASTLE 1.7 SW,#FFFF00,US1WYWS0025
87414,44.1153,-104.6278,UPTON 1.0 NNW,#FFFF00,US1WYWS0026
87415,43.7019,-104.6947,NEWCASTLE 26.2 WSW,#FFFF00,US1WYWS0029


In [24]:
mapmaker_us1.head(100).to_csv('data/mapmaker_100pts.csv')

In [26]:
mapmaker_us1_no_names = mapmaker_us1.copy()
mapmaker_us1_no_names['note'] = mapmaker_us1_no_names['note'] + ' ' + mapmaker_us1_no_names['name']
mapmaker_us1_no_names['name'] = ''

In [27]:
mapmaker_us1_no_names.head(100)

,latitude,longitude,name,color,note
52397,40.5268,-105.1113,,#FFFF00,US10RMHS145 RMHS 1.6 SSW
52398,40.5680,-98.5069,,#FFFF00,US10adam001 JUNIATA 1.5 S
52399,40.5093,-98.5493,,#FFFF00,US10adam002 JUNIATA 6.0 SSW
52400,40.4663,-98.6537,,#FFFF00,US10adam003 HOLSTEIN 0.1 NW
52401,40.4798,-98.4026,,#FFFF00,US10adam004 AYR 3.5 NE
...,...,...,...,...,...
52492,40.8588,-98.9216,,#FFFF00,US10buff005 GIBBON 8.6 NW
52493,40.9223,-99.3868,,#FFFF00,US10buff006 MILLER 0.5 SE
52494,40.9903,-98.8699,,#FFFF00,US10buff007 RAVENNA 3.3 SE
52495,40.9593,-98.9631,,#FFFF00,US10buff008 RAVENNA 5.3 SW


In [28]:
mapmaker_us1_no_names.to_csv('data/mapmaker_all.csv', columns = ['latitude', 'longitude', 'name', 'color', 'note'])

In [29]:
mapmaker_us1_no_names_100 = mapmaker_us1_no_names.sample(n=100)
mapmaker_us1_no_names_100['color'] = 'FFFA00'
mapmaker_us1_no_names_100.to_csv('data/mapmaker_100_random2.csv', columns = ['latitude', 'longitude', 'name', 'color', 'note'])

In [30]:
us1_stations['state'].value_counts()

CO    3394
TX    3376
NC    1597
NE    1534
FL    1295
IL    1239
KS    1236
IN    1224
NM    1120
CA    1112
TN     939
SC     891
MO     863
OR     845
MN     807
AZ     802
WA     779
GA     772
NY     766
MI     624
WY     592
SD     589
PA     586
AL     582
VA     575
OH     549
WI     529
NJ     523
AR     492
OK     450
IA     433
MD     408
KY     358
MA     306
MT     299
MS     276
ME     255
LA     224
ND     211
NV     206
UT     197
NH     196
ID     183
CT     181
VT     156
WV     115
HI      94
DE      87
RI      84
AK      57
DC      12
Name: state, dtype: int64

In [31]:
ghcnd_stations['state'].value_counts()

TX    5002
CO    4161
CA    2790
NC    2143
NE    2097
      ... 
UM      11
MP      11
PW      11
PI       1
SA       1
Name: state, Length: 76, dtype: int64

In [32]:
ghcnd_stations[ghcnd_stations['id'] == 'USW00094008']

,id,latitude,longitude,elevation,state,name,gsn flag,hcn/crn flag,wmo id
113843,USW00094008,48.2138,-106.6213,696.5,MT,GLASGOW INTL AP,GSN,HCN,72768


In [35]:
ghcnd_stations[ghcnd_stations['name'].str.contains('palo alto'.upper())]

,id,latitude,longitude,elevation,state,name,gsn flag,hcn/crn flag,wmo id
40833,MXN00001015,21.9000,-101.9667,2037.3,NaN,PALO ALTO,NaN,NaN,NaN
45009,MXN00026306,27.2833,-109.3500,83.8,NaN,KM. 91 + 600 PALO ALTO,NaN,NaN,NaN
56238,US1CASC0014,37.4190,-122.1214,8.8,CA,PALO ALTO 1.7 NE,NaN,NaN,NaN
56240,US1CASC0017,37.4516,-122.1486,9.8,CA,PALO ALTO 1.2 NE,NaN,NaN,NaN
56244,US1CASC0028,37.4098,-122.1348,18.3,CA,PALO ALTO 0.8 NNE,NaN,NaN,NaN
56471,US1CASM0034,37.4637,-122.1372,4.6,CA,EAST PALO ALTO 0.3 WSW,NaN,NaN,NaN
89375,USC00046642,37.4333,-122.1667,18.0,CA,PALO ALTO,NaN,NaN,NaN
89376,USC00046646,37.4436,-122.1403,7.6,CA,PALO ALTO,NaN,NaN,NaN
96256,USC00226670,33.6500,-88.8500,76.2,MS,PALO ALTO,NaN,NaN,NaN


In [36]:
palo_alto_data = pd.read_csv('data/GHCN_20191001/USC00046642.csv')
palo_alto_data

,STATION,DATE,LATITUDE,LONGITUDE,ELEVATION,NAME,PRCP,PRCP_ATTRIBUTES,SNOW,SNOW_ATTRIBUTES,...,WT05,WT05_ATTRIBUTES,WT08,WT08_ATTRIBUTES,WT11,WT11_ATTRIBUTES,WT14,WT14_ATTRIBUTES,WT16,WT16_ATTRIBUTES
0,USC00046642,1906-03-01,37.43333,-122.16667,18.0,"PALO ALTO, CA US",0.0,"P,,6",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,USC00046642,1906-03-02,37.43333,-122.16667,18.0,"PALO ALTO, CA US",0.0,"P,,6",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,USC00046642,1906-03-03,37.43333,-122.16667,18.0,"PALO ALTO, CA US",0.0,"P,,6",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,USC00046642,1906-03-04,37.43333,-122.16667,18.0,"PALO ALTO, CA US",53.0,",,6",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,USC00046642,1906-03-05,37.43333,-122.16667,18.0,"PALO ALTO, CA US",264.0,",,6",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12209,USC00046642,1953-08-27,37.43333,-122.16667,18.0,"PALO ALTO, CA US",0.0,",,0",0.0,",,0",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12210,USC00046642,1953-08-28,37.43333,-122.16667,18.0,"PALO ALTO, CA US",0.0,",,0",0.0,",,0",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12211,USC00046642,1953-08-29,37.43333,-122.16667,18.0,"PALO ALTO, CA US",0.0,",,0",0.0,",,0",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12212,USC00046642,1953-08-30,37.43333,-122.16667,18.0,"PALO ALTO, CA US",15.0,",,0",0.0,",,0",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [37]:
palo_alto_data.columns

Index(['STATION', 'DATE', 'LATITUDE', 'LONGITUDE', 'ELEVATION', 'NAME', 'PRCP',
       'PRCP_ATTRIBUTES', 'SNOW', 'SNOW_ATTRIBUTES', 'SNWD', 'SNWD_ATTRIBUTES',
       'TMAX', 'TMAX_ATTRIBUTES', 'TMIN', 'TMIN_ATTRIBUTES', 'TOBS',
       'TOBS_ATTRIBUTES', 'WT01', 'WT01_ATTRIBUTES', 'WT03', 'WT03_ATTRIBUTES',
       'WT05', 'WT05_ATTRIBUTES', 'WT08', 'WT08_ATTRIBUTES', 'WT11',
       'WT11_ATTRIBUTES', 'WT14', 'WT14_ATTRIBUTES', 'WT16',
       'WT16_ATTRIBUTES'],
      dtype='object')

In [38]:
palo_alto_data2 = pd.read_csv('data/GHCN_20191001/USC00046646.csv')
palo_alto_data2

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (25,27,29,31) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,STATION,DATE,LATITUDE,LONGITUDE,ELEVATION,NAME,PRCP,PRCP_ATTRIBUTES,SNOW,SNOW_ATTRIBUTES,...,WT01,WT01_ATTRIBUTES,WT03,WT03_ATTRIBUTES,WT04,WT04_ATTRIBUTES,WT05,WT05_ATTRIBUTES,WT11,WT11_ATTRIBUTES
0,USC00046646,1953-09-01,37.4436,-122.1402,7.6,"PALO ALTO, CA US",0.0,",,0,",0.0,",,0",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,USC00046646,1953-09-02,37.4436,-122.1402,7.6,"PALO ALTO, CA US",0.0,",,0,0800",0.0,",,0",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,USC00046646,1953-09-03,37.4436,-122.1402,7.6,"PALO ALTO, CA US",0.0,",,0,0800",0.0,",,0",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,USC00046646,1953-09-04,37.4436,-122.1402,7.6,"PALO ALTO, CA US",0.0,",,0,0800",0.0,",,0",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,USC00046646,1953-09-05,37.4436,-122.1402,7.6,"PALO ALTO, CA US",0.0,",,0,0800",0.0,",,0",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22628,USC00046646,2017-12-18,37.4436,-122.1402,7.6,"PALO ALTO, CA US",0.0,",,7,0800",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22629,USC00046646,2017-12-25,37.4436,-122.1402,7.6,"PALO ALTO, CA US",0.0,",,7,0800",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22630,USC00046646,2017-12-26,37.4436,-122.1402,7.6,"PALO ALTO, CA US",0.0,",,7,0800",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22631,USC00046646,2017-12-29,37.4436,-122.1402,7.6,"PALO ALTO, CA US",0.0,",,7,0800",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Rough data size for __USC__ data

In [40]:
# 100*365 = 36500 rows for 100 years of daily data
# 22480 USC data stations
22480*36500

820520000

### Approx 0.8 billion rows is a lot of data to start with.

To simplify the problem and allow the development of a prototype visualization a couple of approaches could be taken:
1. Focus on data from a list of cities, such as [Natural Earth Cities])http://www.naturalearthdata.com/downloads/10m-cultural-vectors/)
    * Then use an algorithm (or perhaps just pick the largest file) to select a data station for a given city within a certain radius of that city
2. Only focus on a region of interest that is smaller than the full US (e.g. California)

### Next Steps
* Build a GIS database using e.g. PostGIS to allow easy queries of the _stations_ data
* Select a subset of U.S. stations and load those data into the database